In [1]:
!pip install pyngrok
!ngrok config add-authtoken <TOKEN>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
%%writefile main.py

import json
import random
from http.server import BaseHTTPRequestHandler, HTTPServer

class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/plain')
        self.end_headers()
        self.wfile.write(b'OK')

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length).decode('utf-8')
        response_data = self.process_post_data(post_data)

        self.send_response(200)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        self.wfile.write(response_data)

    def process_post_data(self, post_data):
        vulnerable = random.choices([True, False], weights=[3, 97])[0]

        response_data = json.dumps({'vulnerable': vulnerable}).encode('utf-8')
        return response_data

def run_server():
    server_address = ('', 8000)
    httpd = HTTPServer(server_address, SimpleHTTPRequestHandler)
    httpd.serve_forever()

if __name__ == '__main__':
    run_server()


Overwriting main.py


In [ ]:
import os
import asyncio

async def run_process(cmd):
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            decoded_line = line.strip().decode('utf-8')
            if 'ngrok' in decoded_line and "url" in decoded_line:
                model_endpoint_url = decoded_line.split("url=")[-1]
                print(f"MODEL_ENDPOINT_URL={model_endpoint_url}")

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

await asyncio.gather(
    run_process(['python', 'main.py']),
    run_process(['ngrok', 'http', '--log', 'stderr', '8000']),
)


MODEL_ENDPOINT_URL=https://6e60-34-145-121-48.ngrok-free.app
